In [ ]:
!pip install -q -U google-generativeai

In [ ]:
from google.colab import userdata
import google.generativeai as genai
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

: 

In [ ]:
from google.colab import userdata
import google.generativeai as genai
import os
from IPython.display import Markdown

!pip install -q -U google-generativeai

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Replace with the actual path to your video file
video_file_name = "media/videos/plate_number_video.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")

import time

# Check whether the file is ready to be used.
while video_file.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)

# Create the prompt.  This is the key change for the new task
prompt = """
            Please analyze the image of a vehicle's license plate and output the following details in JSON format:
            {
                'plate': 'B 1234 ABC',
                'vehicle': 'car',
                'type': 'sedan',
                'color': 'red',
                'expiry_date': '07 - 17',
                'gate_open_time': '2024-12-02 18:15:01',
                'gate_closed_time': 'N/A'
            }. Only return the JSON object, and use 'N/A' for missing or unknown information.
            """

response = model.generate_content([video_file, prompt],
                                  request_options={"timeout": 600})

# Print the response, rendering any Markdown
Markdown(response.text)

```json
{
    'plate': 'B 1308 RFK',
    'vehicle': 'car',
    'type': 'SUV',
    'color': 'black',
    'expiry_date': '09-22',
    'gate_open_time': 'N/A',
    'gate_closed_time': 'N/A'
}
```